In [51]:
# lstm for time series forecasting
from numpy import sqrt, asarray
from pandas import read_csv
import tensorflow as tf
from tensorflow import keras, math, reduce_mean
from keras import Sequential
from keras.layers import Dense, LSTM, BatchNormalization
import time
from datetime import datetime
import sys

In [52]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps=5):
    X, y = list(), list()
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        # filter this properly
        seq_x, seq_y = sequence[i:end_ix+1, :-1], sequence[i:end_ix+1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return asarray(X), asarray(y)

# Need new function to evaluate RMSE based on just 'Percent Output'
# def my_loss_fn(y_actual, y_pred):
#     SE = math_ops.squared_difference(y_pred.numpy()[0], y_actual.numpy()[0])  #squared difference
#     MSE = sqrt(SE)
#     return MSE

In [53]:
# a = keras.backend.constant([1,2,3])
# b = keras.backend.constant([4,5,6])

# loss = my_loss_fn(a,b)
# print(loss)

# load the dataset
df = read_csv("SolarTrainingData.csv", index_col=0)

In [54]:
df = df.drop(['year', 'day'], axis=1)
df.tail()

,temp,feelslike,dew,humidity,precip,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,month,time,Percent Output
73657,97.4,99.1,62.8,32.19,0.0,0,0,16.1,10.6,138.0,1009.6,11.3,9.9,405.0,1.5,4.0,8,18.0,0.106900
73660,97.4,99.1,62.8,32.19,0.0,0,0,16.1,10.6,138.0,1009.6,11.3,9.9,405.0,1.5,4.0,8,18.0,0.295373
73663,97.4,99.1,62.8,32.19,0.0,0,0,16.1,10.6,138.0,1009.6,11.3,9.9,405.0,1.5,4.0,8,18.0,0.022917
73664,97.4,99.1,62.8,32.19,0.0,0,0,16.1,10.6,138.0,1009.6,11.3,9.9,405.0,1.5,4.0,8,18.0,0.346222
73666,97.4,99.1,62.8,32.19,0.0,0,0,16.1,10.6,138.0,1009.6,11.3,9.9,405.0,1.5,4.0,8,18.0,0.236087


In [80]:
# retrieve the values
values = df.values.astype('float32')
# specify the window size
n_steps = 5
# split into samples
X, y = split_sequence(values, n_steps)
# reshape into [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], X.shape[2]))
# split into train/test
n_test = int(X.shape[0]/3)
X_train, X_test, y_train, y_test = X[:-n_test], X[-n_test:], y[:-n_test], y[-n_test:]

[[[ 25.   14.1  11.4 ...   0.    1.    8. ]
  [ 25.   14.1  11.4 ...   0.    1.    8. ]
  [ 25.   14.1  11.4 ...   0.    1.    8. ]
  [ 25.   14.1  11.4 ...   0.    1.    8. ]
  [ 25.   14.1  11.4 ...   0.    1.    8. ]
  [ 25.   14.1  11.4 ...   0.    1.    8. ]]

 [[ 25.   14.1  11.4 ...   0.    1.    8. ]
  [ 25.   14.1  11.4 ...   0.    1.    8. ]
  [ 25.   14.1  11.4 ...   0.    1.    8. ]
  [ 25.   14.1  11.4 ...   0.    1.    8. ]
  [ 25.   14.1  11.4 ...   0.    1.    8. ]
  [ 25.   14.1  11.4 ...   0.    1.    8. ]]

 [[ 25.   14.1  11.4 ...   0.    1.    8. ]
  [ 25.   14.1  11.4 ...   0.    1.    8. ]
  [ 25.   14.1  11.4 ...   0.    1.    8. ]
  [ 25.   14.1  11.4 ...   0.    1.    8. ]
  [ 25.   14.1  11.4 ...   0.    1.    8. ]
  [ 25.   14.1  11.4 ...   0.    1.    8. ]]

 ...

 [[ 98.7 100.1  61.8 ...   6.    7.   13. ]
  [ 98.7 100.1  61.8 ...   6.    7.   13. ]
  [ 98.7 100.1  61.8 ...   6.    7.   13. ]
  [ 98.7 100.1  61.8 ...   6.    7.   13. ]
  [ 98.7 100.1  61.8

In [84]:
# define model
# improvement area : try adding dropout
model = Sequential()
#model.add(BatchNormalization())
model.add(LSTM(50, activation='relu', kernel_initializer='he_normal', input_shape=(n_steps+1,len(df.iloc[0])-1)))
model.add(BatchNormalization())
model.add(Dense(1))
# compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [85]:
# fit the model
model.fit(X_train, y_train, epochs=120, batch_size=32, verbose=2, validation_data=(X_test, y_test))
# evaluate the model
mse, mae = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f, MAE: %.3f' % (mse, sqrt(mse), mae))

Epoch 1/70
193/193 - 6s - loss: 0.1666 - mae: 0.2781 - val_loss: 0.0522 - val_mae: 0.1788 - 6s/epoch - 30ms/step
Epoch 2/70
193/193 - 2s - loss: 0.0435 - mae: 0.1587 - val_loss: 0.0412 - val_mae: 0.1612 - 2s/epoch - 8ms/step
Epoch 3/70
193/193 - 2s - loss: 0.0310 - mae: 0.1379 - val_loss: 840.8117 - val_mae: 0.6825 - 2s/epoch - 9ms/step
Epoch 4/70
193/193 - 2s - loss: 0.0278 - mae: 0.1297 - val_loss: 7643.3687 - val_mae: 2.3567 - 2s/epoch - 9ms/step
Epoch 5/70
193/193 - 2s - loss: 0.0276 - mae: 0.1290 - val_loss: 0.5156 - val_mae: 0.1795 - 2s/epoch - 11ms/step
Epoch 6/70
193/193 - 2s - loss: 0.0345 - mae: 0.1417 - val_loss: 6806.5483 - val_mae: 2.2589 - 2s/epoch - 9ms/step
Epoch 7/70
193/193 - 2s - loss: 0.0258 - mae: 0.1255 - val_loss: 2632.7729 - val_mae: 1.0979 - 2s/epoch - 8ms/step
Epoch 8/70
193/193 - 2s - loss: 0.0255 - mae: 0.1248 - val_loss: 0.0470 - val_mae: 0.1723 - 2s/epoch - 8ms/step
Epoch 9/70
193/193 - 1s - loss: 0.0239 - mae: 0.1211 - val_loss: 0.0368 - val_mae: 0.1571 -

In [ ]:
# make a prediction
input = df.iloc[-n_steps-1:]
input = input.drop('Percent Output', axis=1)
input = asarray(input).reshape(1, n_steps+1, 18)
print(model.predict(input))
#row = asarray(df.iloc[-1])
#yhat = model.predict(row)
#print('Predicted: %.3f' % (yhat))

1/1 [==============================] - 0s 324ms/step
[[0.40301567]]
